In [21]:
print(Sys.time())

library(dplyr)
library(Seurat)
library(ggplot2)
library(gplots)
library(NMF)
library(registry)
library(rngtools)
library(purrr)
library(cowplot)
library(stringr)
library(pkgmaker)
library(cluster)

#source("/KINOMO/KINOMO.R")

pat <- 'Filename'

Filename <- '../../../CompiledData/QCdata/SarcomaS914GEX/data_SarcomaS914GEX_genes_300_UMI_600'
pat ='SarcomaS914GEX'
integrated<-readRDS(paste0(Filename,".rds"))
#saveRDS(integrated, "S914GEX2000CellSample.rds")


#Seurat Object creation and necessary pre-filteration, normalization. Please customize if necessary.
DefaultAssay(object = integrated) <- "RNA"
integrated <- integrated[, sample(colnames(integrated), size =2000, replace=F)]



[1] "2022-11-04 12:30:03 UTC"


In [24]:
seu.1<-integrated

mito.genes <- grep(pattern = "^MT-", x = rownames(seu.1), value = TRUE)
rbl.genes <- grep(pattern = "^RB-", x = rownames(seu.1), value = TRUE)
rsl.genes <- grep(pattern = "^RS-", x = rownames(seu.1), value = TRUE)
rpl.genes <- grep(pattern = "^RPL-", x = rownames(seu.1), value = TRUE)
rbl.genes <- grep(pattern = "^RBL-", x = rownames(seu.1), value = TRUE)
rps.genes <- grep(pattern = "^RPS-", x = rownames(seu.1), value = TRUE)
rbs.genes <- grep(pattern = "^RBS-", x = rownames(seu.1), value = TRUE)
rbl1.genes <- grep(pattern = "^RB", x = rownames(seu.1), value = TRUE)
rsl1.genes <- grep(pattern = "^RS", x = rownames(seu.1), value = TRUE)
rpl1.genes <- grep(pattern = "^RPL", x = rownames(seu.1), value = TRUE)
rbl1.genes <- grep(pattern = "^RBL", x = rownames(seu.1), value = TRUE)
rps1.genes <- grep(pattern = "^RPS", x = rownames(seu.1), value = TRUE)
rbs1.genes <- grep(pattern = "^RBS", x = rownames(seu.1), value = TRUE)

counts <- GetAssayData(seu.1, assay = "RNA")
counts <- counts[-(which(rownames(counts) %in% c(mito.genes,rbl.genes,rsl.genes,rpl.genes,rbl.genes,rps.genes,rbs.genes,rbl1.genes,rsl1.genes,rpl1.genes,rbl1.genes,rps1.genes,rbs1.genes))),]
seu <- subset(seu.1, features = rownames(counts))

seu <- NormalizeData(seu, normalization.method = "LogNormalize", scale.factor = 10000)
seu <- FindVariableFeatures(seu, selection.method = "vst", nfeatures = 2000)
seu <- ScaleData(seu, features = rownames(seu))
seu <- RunPCA(seu, features = VariableFeatures(object = seu))
seu <- FindNeighbors(seu, dims = 1:15)
seu <- FindClusters(seu,resolution=0.4)
seu <- RunUMAP(seu, dims = 1:20)

mat<-as.matrix(seu@assays$RNA@data)
mat<-mat[rowSums(mat)>0,]

#Estimating the factorization rank
#A critical parameter in KINOMO-NMF is the factorization rank r. It defines the number of metagenes used to approximate the target matrix. Given a 
#NMF method and 
#the target matrix, a common way of deciding on r is to try different values, compute some quality measure of the results, and choose the best value 
#according to this quality criteria.
#Several approaches have then been proposed to choose the optimal value of r. For example, (Brunet2004) proposed to take the first value of r 
#for which the cophenetic coefficient #starts decreasing, (Hutchins2008) suggested to choose the first value where the RSS curve presents an 


Centering and scaling data matrix

PC_ 1 
Positive:  TIMP3, COL4A1, FAP, PRUNE2, ADAMTSL1, SDK1, ANXA1, DUXAP8, LINC02476, GPSM2 
	   TRABD2B, NLGN1, STEAP4, PLA2G2A, SMC4, MAGI2, PLPP1, PDE10A, HIST1H2AG, AF165147.1 
	   KCND2, GALNT13, NDE1, PAPPA, PDE3A, VCL, THBS2, IGFBP7, HIST1H4H, HIST1H2AC 
Negative:  ITPR2, IQGAP2, STAB1, SFMBT2, FLI1, RBM47, F13A1, DOCK8, DOCK2, DLEU2 
	   MRC1, FMN1, KCNMA1, FHIT, DAPK1, MEF2C, CR1, CMIP, MS4A6A, CHST11 
	   LINC00278, FRMD4B, CD163, MCTP1, SELENOP, SIGLEC1, MGAT4A, SLCO2B1, NPL, AL162414.1 
PC_ 2 
Positive:  MRC1, RBM47, DOCK2, AL162414.1, SIGLEC1, F13A1, CD163, CR1, FMN1, IL10RA 
	   KCNMA1, TLR2, MS4A4A, MS4A6A, CYBB, NPL, LILRB5, CPVL, SLCO2B1, MS4A7 
	   FYB1, ME1, AOAH, SAMSN1, CYTH4, CSF1R, PRKCB, MS4A4E, SLC1A3, SYK 
Negative:  ADGRL4, VWF, PREX2, RAPGEF5, KALRN, LDB2, GALNT18, CDH5, FLT1, ANGPT2 
	   DIPK2B, THSD7A, MECOM, SHANK3, SPRY1, ANO2, EFNB2, RASGRF2, ADAMTS9, IGFBP3 
	   KDR, TCIM, EMCN, NOTCH4, MCAM, CHRM3, ARHGAP29, ARHGEF

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2000
Number of edges: 60168

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8784
Number of communities: 9
Elapsed time: 0 seconds


12:38:09 UMAP embedding parameters a = 0.9922 b = 1.112

12:38:09 Read 2000 rows and found 20 numeric columns

12:38:09 Using Annoy for neighbor search, n_neighbors = 30

12:38:09 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

12:38:09 Writing NN index file to temp file /tmp/RtmpjGa7kl/file51cd7e94e67

12:38:09 Searching Annoy index using 1 thread, search_k = 3000

12:38:10 Annoy recall = 100%

12:38:10 Commencing smooth kNN distance calibration using 1 thread

12:38:11 Initializing from normalized Laplacian + noise

12:38:11 Commencing optimization for 500 epochs, with 81168 positive edges

12:38:13 Optimization finished



In [25]:
#inflection point, and (Frigyesi2008) considered the smallest value at which #the decrease in the RSS is lower than the decrease of the RSS obtained 
#from random data.

pdf(file = paste0(pat,"_KINOMO_nmf.pdf"),height = 20,width=20)
#nmf.estimate.rank <- KINOMO(mat, 2:10, nrun=10)
nmf.estimate.rank <- nmf(mat, 2:3, nrun=5)
#saveRDS(nmf.estimate.rank, paste0(pat,"/",pat,"_san_check_KINOMO_nmf_rank_temp_.rds"))
plot(nmf.estimate.rank)
dev.off()

png 
  2

In [23]:
integrated@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rps,percent.rpl,percent.rp,patient,celltype_initial_singleR,predicted_doublets,doublet_scores,RNA_snn_res.0.8,seurat_clusters,old.ident,celltype_bped_fine
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<dbl>,<fct>,<fct>,<fct>,<fct>
TCAGCAACAAGCTGTT-1,SarcomaS914GEX,14075,4879,1.07992895,0.25577265,0.3410302,0.59680284,SarcomaS914GEX,Fibroblasts,FALSE,0.08411349,1,1,1,Fibroblasts
ACTTTCAGTGGTAACG-1,SarcomaS914GEX,2207,1438,7.88400544,0.36248301,0.2265519,0.58903489,SarcomaS914GEX,Fibroblasts,FALSE,0.04964773,2,2,2,Fibroblasts
TCGTAGACACATCTTT-1,SarcomaS914GEX,6047,2911,2.03406648,0.41342815,0.4630395,0.87646767,SarcomaS914GEX,Fibroblasts,FALSE,0.16349727,1,1,1,Fibroblasts
ACACCAAAGAGAACAG-1,SarcomaS914GEX,716,570,2.09497207,0.13966480,0.0000000,0.13966480,SarcomaS914GEX,Adipocytes,FALSE,0.05405714,2,2,2,Fibroblasts
ATGGGAGTCAGGTTCA-1,SarcomaS914GEX,789,540,4.94296578,0.88719899,1.0139417,1.90114068,SarcomaS914GEX,Fibroblasts,FALSE,0.03731747,2,2,2,Fibroblasts
TAGTGGTTCAGTCCCT-1,SarcomaS914GEX,943,725,4.13573701,0.42417815,0.4241782,0.84835631,SarcomaS914GEX,Fibroblasts,FALSE,0.03483685,2,2,2,Fibroblasts
CAGTAACCAGCCTTGG-1,SarcomaS914GEX,2078,1491,1.29932628,0.81809432,0.6737247,1.49181906,SarcomaS914GEX,Endothelial cells,FALSE,0.17678344,11,11,11,Adipocytes
CCGTACTCACACCGCA-1,SarcomaS914GEX,6773,2731,0.08858704,0.05905802,0.0000000,0.05905802,SarcomaS914GEX,Macrophages,FALSE,0.03324832,4,4,4,Macrophages
CTAGCCTCAGTCTTCC-1,SarcomaS914GEX,1653,1051,3.02480339,0.24198427,0.1209921,0.36297641,SarcomaS914GEX,CD8+ T-cells,FALSE,0.19709310,9,9,9,Adipocytes


In [20]:

integrated@meta.data

#TGGCCAGTCCCACTTG-1

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rps,percent.rpl,percent.rp,patient,celltype_initial_singleR,predicted_doublets,doublet_scores,RNA_snn_res.0.8,seurat_clusters,old.ident,celltype_bped_fine
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<dbl>,<fct>,<fct>,<fct>,<fct>
TGGCCAGTCCCACTTG-1,SarcomaS914GEX,15596,5914,0.6283662,0.1474737,0.23082842,0.3783021,SarcomaS914GEX,Fibroblasts,FALSE,0.27571615,3,3,3,Fibroblasts
AACTCTTCATATACCG-1,SarcomaS914GEX,7982,3778,2.3552994,0.3507893,0.38837384,0.7391631,SarcomaS914GEX,Fibroblasts,FALSE,0.17218693,8,8,8,Fibroblasts
TCAATCTGTCGAGTTT-1,SarcomaS914GEX,4759,2569,2.2063459,0.7144358,0.73544862,1.4498844,SarcomaS914GEX,Fibroblasts,FALSE,0.12514790,14,14,14,Fibroblasts
TAAGAGAAGTTGAGAT-1,SarcomaS914GEX,6983,3434,0.9881140,0.1288844,0.12888443,0.2577689,SarcomaS914GEX,Fibroblasts,FALSE,0.24996238,3,3,3,Fibroblasts
GCAATCACAACTGCTA-1,SarcomaS914GEX,2935,1771,0.6814310,0.7836457,1.22657581,2.0102215,SarcomaS914GEX,Chondrocytes,FALSE,0.10677855,7,7,7,Fibroblasts
ACATACGTCTGCCCTA-1,SarcomaS914GEX,1082,810,1.2014787,0.0000000,0.92421442,0.9242144,SarcomaS914GEX,Fibroblasts,FALSE,0.02806248,6,6,6,Fibroblasts
TCACGAAGTGGGTATG-1,SarcomaS914GEX,8328,3871,0.7444765,0.1200768,0.01200768,0.1320845,SarcomaS914GEX,Fibroblasts,FALSE,0.10918342,3,3,3,Fibroblasts
TAAGCGTAGAGGTACC-1,SarcomaS914GEX,14310,4865,1.0412299,0.6429071,0.93640811,1.5793152,SarcomaS914GEX,Fibroblasts,FALSE,0.11681755,0,0,0,Fibroblasts
CTCGGAGAGTGTACGG-1,SarcomaS914GEX,9241,3964,0.5627097,0.3679255,0.66010172,1.0280273,SarcomaS914GEX,Fibroblasts,FALSE,0.21470588,8,8,8,Fibroblasts


In [11]:
print(Sys.time())

library(dplyr)
library(Seurat)
library(ggplot2)
library(gplots)
library(NMF)
library(registry)
library(rngtools)
library(purrr)
library(cowplot)
library(stringr)
library(pkgmaker)
library(cluster)

#source("/KINOMO/KINOMO.R")

pat <- 'Filename'

Filename <- '../../../CompiledData/QCdata/SarcomaS410GEX/data_SarcomaS410GEX_genes_300_UMI_600'
pat ='SarcomaS410GEX'
integrated2<-readRDS(paste0(Filename,".rds"))
DefaultAssay(object = integrated2) <- "RNA"



[1] "2022-11-04 12:22:50 UTC"


In [12]:
integrated2

An object of class Seurat 
24008 features across 1927 samples within 1 assay 
Active assay: RNA (24008 features, 2000 variable features)
 2 dimensional reductions calculated: pca, umap